# MSC Geomet - Using Authentication to Access Flow Predictions
## Quick Demo - WCS

This notebook is designed to quickly help users learn about user/password authentication on the Meteorological Service of Canada's (MSC) [GeoMet](https://eccc-msc.github.io/open-data/msc-geomet/readme_en/#msc-geomet "MSC GeoMet Open Documentation") platform, in order to access flow predictions. We can access the flow predictions through the web map service (WMS) and web coverage service (WCS) with Python. If you are unsure of which service to use, a general rule of thumb is that the <b style="color:Orange;">web map service</b> is primarily used to retrieve <b style="color:Orange;">geospatial images</b> and the <b style="color:Violet;">web coverage service</b> is used to retrieve <b style="color:Violet;">raw geospatial data</b>. This tutorial demonstrates examples using the <b style="color:Violet;">web coverage service</b>. If you are new to these services we recommend later visiting the more comprehensive tutorials in this repository for accessing flow data with the WMS and WCS. 

## Table of Contents

* [What pre-requistes do I need for this tutorial?](#What-pre-requistes-do-I-need-for-this-tutorial?)
* [How do I get to GeoMet? How do I start retrieving data from GeoMet?](#How-do-I-get-to-GeoMet?-How-do-I-start-retrieving-data-from-GeoMet?)
    * [Establish a connection to GeoMet](#Establish-a-connection-to-GeoMet)
    * [Check what data is available](#Check-what-data-is-available)
    * [Check what forecast times are available](#Check-what-forecast-times-are-available)
    * [Making a simple request with WCS](#Making-a-simple-request-with-WCS)


## What pre-requistes do I need for this tutorial?

First, we import all necessary Python libraries.  The libraries needed are standard Python libraries. The library that we'll use to access GeoMet is a third-party library, [OWSLib](https://geopython.github.io/OWSLib/index.html "OWSLib Documentation"), which stands for the Open Geospatial Consortium (OGC) web service. [Xarray](https://docs.xarray.dev/en/stable/ "xarray Documentation") is another third party library that we recommend for handling the requested data.

In [1]:
# data
import warnings
import re
import time
import configparser
from datetime import datetime, timedelta
from IPython.display import Image

import xarray as xr

# web map services 
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
from owslib.wcs import Authentication

To access the flow prediction from the user/password authenticated version of GeoMet, we need credentials. We can use the built-in configparser utility library, which reads a file with the following format:

```
[Login]
Username = your_user_name 
Password = your_password
```

In [2]:
config = configparser.ConfigParser()
config.read_file(open('config.cfg'))

login = config['Login']

## How do I get to GeoMet? How do I start retrieving data from GeoMet?


When we want to get prediction data from GeoMet we will typically follow three steps:

1. Establish a connection with the GeoMet server. GeoMet will send back a response if we connect successfully.
2. Look "under-the-hood" of the GeoMet response to see what data is available* and for what times.
3. Use our GeoMet connection that we made in step 1, to request the data we are interested in.

*Step 2 is slightly different for authenticated data. We will highlight the differences below.

### Establish a connection to GeoMet

In [3]:
# filter warnings or be prepared to see plenty
warnings.filterwarnings('ignore', module='owslib', category=UserWarning)

We are going to first connect to the WMS to look at the data available and its metadata. Then, we will switch to the WCS to access the predictions.

In [4]:
# connect to the web map service
wms = WebMapService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WMS',
                    version='1.3.0',
                    timeout=300)

### Check what data is available

In [5]:
# let's have a look at the first 20 data layers in the wms contents
for key in list(wms.contents.keys())[0:20]:
    print(key)

Canadian Weather
ALERTS
CURRENT_CONDITIONS
METNOTES
Regional Deterministic Prediction System (RDPS) [10 km]
RDPS - Coupled to Gulf of St. Lawrence (RDPS-CGSL)
CGSL.ETA_ICEC
CGSL.ETA_ICEPRS
CGSL.ETA_ICESTG
CGSL.ETA_ICET
CGSL.ETA_ICETK
CGSL.ETA_UICE
CGSL.ETA_UOGRD
CGSL.ETA_UU
CGSL.ETA_WTMP
RDPS convective fields
RDPS.CONV_KINDEX.PT3H
RDPS.CONV_ML-CAPE
RDPS.CONV_ML-CIN
RDPS.CONV_ML-EL-HGT


This list tells us what the layer names are available, but they aren't very descriptive. Let's print their titles instead.

In [6]:
# let's have a look at the descriptive names
for key in list(wms.contents.keys())[0:20]:
    print(wms[key].title)

Canadian Weather
Weather Alerts [experimental]
Current Conditions
MetNotes
Regional Deterministic Prediction System (RDPS) [10 km]
RDPS - Coupled to Gulf of St. Lawrence (RDPS-CGSL)
CGSL.ETA.ICEC - Ice cover fraction
CGSL.ETA.ICEPRS - Vertically integrated ice internal pressure (or stress) [N/m]
CGSL.ETA.ICESTG - Vertically integrated compressive ice strength [N/m]
CGSL.ETA.ICET - Ice surface temperature [K]
CGSL.ETA.ICETK - Ice thickness [m]
CGSL.ETA.UICE - Ice speed vector [m/s]
CGSL.ETA.UOGRD - Surface ocean current [m/s]
CGSL.ETA.UU - Wind at surface level [m/s]
CGSL.ETA.WTMP - Water temperature [K]
RDPS convective fields
RDPS.CONV - Thunderstorm potential index George K (3 hourly forecast)
RDPS.CONV - CAPE for a parcel representing the mean of a layer [J/kg]
RDPS.CONV - CIN for a parcel representing the mean of a layer [J/kg]
RDPS.CONV - Height of the equilibrium level for a parcel representing the mean of a layer [m AGL]


Password protected data available on GeoMet will not appear in the lists above. The names of the layers are hidden by design. We must know the name of the hidden layer we are interested ahead of time. 

In [7]:
# for this example, let's have a look at DHPS streamflow
layer_name = 'DHPS_1km_RiverDischarge'

In [8]:
# re-establish a connection with GeoMet, this time specifying the layer and providing our "login" credentials
wms_auth = WebMapService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WMS&LAYERS={layer_name}',
                        version='1.3.0',
                        auth=Authentication(username=login['Username'], password=login['Password']),
                        timeout=300)

In [9]:
for key in list(wms_auth.contents.keys()):
    print(key)

Deterministic Hydrological Prediction System (DHPS) [1 km]
DHPS
DHPS_1km_RiverDischarge


### Check what forecast times are available

In [11]:
wms_auth[layer_name].dimensions

{'time': {'units': 'ISO8601',
  'default': '2022-05-26T13:00:00Z',
  'nearestValue': '0',
  'values': ['2022-05-26T01:00:00Z/2022-06-01T00:00:00Z/PT1H']},
 'reference_time': {'units': 'ISO8601',
  'default': '2022-05-26T00:00:00Z',
  'multipleValues': '1',
  'nearestValue': '0',
  'values': ['2022-05-24T12:00:00Z/2022-05-26T00:00:00Z/PT12H']}}

<b>Reference time [ first available forecast issue / last available forecast issue / time between forecast issues ]</b>

In [13]:
wms_auth[layer_name].dimensions['reference_time']['values']

['2022-05-24T12:00:00Z/2022-05-26T00:00:00Z/PT12H']

<b>Time [ first time for most recent forecast / last time for most recent forecast / forecast timesteps ]</b>

In [28]:
wms_auth[layer_name].dimensions['time']['values']

['2022-05-25T01:00:00Z/2022-05-31T00:00:00Z/PT1H']

### Make a simple WCS request

In [14]:
layer_name = 'DHPS_1km_RiverDischarge'

# connect to the WCS
wcs_auth = WebCoverageService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WCS&COVERAGEID={layer_name}', 
                        auth=Authentication(username=login['Username'], password=login['Password']),
                        version='2.0.1',
                        timeout=300
                        )

In [15]:
response = wcs_auth.getCoverage(identifier = [layer_name], 
                                format = 'image/netcdf', 
                                subsettingcrs = 'EPSG:4326', # default and recommended crs
                                subsets = [('lat', 45.0, 46.0), ('lon', -70.0, -69.0)],
                                DIM_REFERENCE_TIME='2022-05-26T00:00:00Z', # capitalization here is important
                                TIME='2022-05-26T01:00:00Z' # capitalization here is important
                               )

In [16]:
# read into an xarray
ds = xr.open_dataset(response.read()).load()
# print the header for the DataArray
ds.head

<bound method Dataset.head of <xarray.Dataset>
Dimensions:  (lat: 120, lon: 120)
Coordinates:
  * lat      (lat) float64 45.0 45.01 45.02 45.03 ... 45.97 45.98 45.99 46.0
  * lon      (lon) float64 -70.0 -69.99 -69.98 -69.97 ... -69.02 -69.01 -69.0
Data variables:
    Band1    (lat, lon) float32 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    GDAL_TIFFTAG_RESOLUTIONUNIT:  2
    GDAL_TIFFTAG_XRESOLUTION:     72.0
    GDAL_TIFFTAG_YRESOLUTION:     72.0
    Conventions:                  CF-1.5
    GDAL:                         GDAL 3.1.3, released 2020/09/01
    history:                      Thu May 26 13:41:44 2022: GDAL CreateCopy( ...>